### Load shelve into query list

In [45]:
from os import path

def load_query_list(shelf_path='./obj/query_shelf'):
    """Loads a Python shelve file. 
    Resets 'matches' and 'processed' if on different system."""
    with shelve.open(shelf_path, 'r') as shelf:
        query_obj_list = [shelf[key] for key in shelf.keys()]
        
    # Clean up matches if running on different machine
    clean = False
    for query in query_obj_list:
        if query.match:
            for key, match in query.match.items():
                if not path.exists(key):
                    clean = True  # means some directory didn't match up
    # Have to run through after all have been checked
    if clean:
        for query in query_obj_list:
            query.match = None
            query.processed = False
        
    return query_obj_list

query_obj_list = load_query_list()
query_obj_list

### Process query list into table for GUI

In [58]:
import shelve
# Import Classes needed
from milnlp.collection.topic_model import SimpleQuery, ComplexQuery

attribute_list = ['UUID', 'phrase', 'flags']
flag_list = list(query_obj_list[0].flags.keys())

def process_query_list(query_obj_list, attribute_list, flag_list):
    """Takes a query object list and returns the formatted row for the GUI"""         
    query_list = []
    for query in query_obj_list:
        row = []
        for attr in attribute_list:
            if not attr == 'flags':
                row.append(getattr(query, attr))
            else:
                for flag in flag_list:
                    if getattr(query, 'type') == 'simple':
                        row.append(u'\u2713') if getattr(query, attr)[flag] else row.append(u'\u2715')
                    else:
                        row.append('-')
        query_list.append(row)
    return query_list

query_list = process_query_list(query_obj_list, attribute_list, flag_list)
query_list

[['SQ_001', 'hypersonic weapon', '✕', '✕', '✓', '✓', '✓'],
 ['SQ_002', 'hypersonic weapon', '✕', '✕', '✓', '✓', '✕'],
 ['CQ_001', 'SQ_001 or SQ_002', '-', '-', '-', '-', '-'],
 ['CQ_002', 'SQ_001 and SQ_002', '-', '-', '-', '-', '-']]

In [59]:
import pandas as pd

df = pd.DataFrame(query_list)
df.columns = [label.title() for label in attribute_list[:-1]+flag_list]
df

,Uuid,Phrase,Case-Sensitive,Whole-Word,Special-Delims,Plurals,Constituents
0,SQ_001,hypersonic weapon,✕,✕,✓,✓,✓
1,SQ_002,hypersonic weapon,✕,✕,✓,✓,✕
2,CQ_001,SQ_001 or SQ_002,-,-,-,-,-
3,CQ_002,SQ_001 and SQ_002,-,-,-,-,-
